<h1>Web scraping  Higg MSI Index</h1>

<h3>Import necessary modules</h3>

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import requests

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [6]:
#these are the manually collected index codes
textile_url =['211', '199', '194', '200', '203', '204',
           '201', '205' ,'202', '11932', '11933', '210', '209',
          '196','195', '198', '197', '11934', '11931', '207', '208', '206']

synthetic_leather_url = ['193']

leather_url = ['148', '149', '151', '150', '13148']

plastic_url = ['178', '168', '173', '170', '179', '169', '172', '171', 
               '175', '180', '166', '174',  '181', '166', '174', '181', '167', '177']

elastomers_url = ['188', '187', '186', '184', '185', '182', '190', 
                  '183', '192', '11930', '191', '2954']

metals_url = ['161', '160', '165', '163', '153', '155', '154', '164', 
             '162', '152', '158', '159', '156']

wood_based_url = ['212','213']

insulation_url = ['147', '146', '144', '145']

coating_lamination_url = ['138', '137', '136' , '135', '140', '139']

foam_url = ['141', '142', '143', '7966']

all_materials = [textile_url, synthetic_leather_url, leather_url, plastic_url,
                 elastomers_url, metals_url, wood_based_url, insulation_url,
                 coating_lamination_url, foam_url]

In [2]:
keyword = '1'
url = 'https://msi.higg.org/process/' + keyword

In [83]:
def get_indexdata(url, keyword):
    
    # create a new Firefox session
    driver = webdriver.Chrome('/Users/Jeremy/Documents/Columbia MS&E/Summer Startup Track 2020/Higg-MSI/chromedriver')
    driver.implicitly_wait(30)
    driver.get(url+keyword)
    
    driver.find_element_by_xpath("""//*[@id="content-header"]/div[3]/div[1]/span[2]""").click()
    

    #Selenium hands the page source to Beautiful Soup
    results_page = BeautifulSoup(driver.page_source, 'lxml')
    
    
    name = results_page.find_all('div', class_ = 'header-title')
    material_name_list = []
   
    
    for name_1 in name:
        try:
            name_1 = name_1.text.strip()
            material_name_list.append(name_1)
        except:
            pass
        
    material_name = material_name_list[0]
    
    
    chart1 = results_page.find_all('div', class_='chart-section chart-section__1 tooltipstered')
    chart2 = results_page.find_all('div', class_='chart-section chart-section__2 tooltipstered')
    chart3 = results_page.find_all('div', class_='chart-section chart-section__3 tooltipstered')
    chart5 = results_page.find_all('div', class_='chart-section chart-section__5 tooltipstered')
    chart6 = results_page.find_all('div', class_='chart-section chart-section__6 tooltipstered')
    
    

    try: 
        start = chart1[1]['data-tooltip'].find("MSI score: ") + len("MSI score: ")
        end = chart1[1]['data-tooltip'].find("points")

        GlobalWarming = chart1[1]['data-tooltip'][start:end]

        start = chart2[1]['data-tooltip'].find("MSI score: ") + len("MSI score: ")
        end = chart2[1]['data-tooltip'].find("points")

        Eutrophication = chart2[1]['data-tooltip'][start:end]

        start = chart3[1]['data-tooltip'].find("MSI score: ") + len("MSI score: ")
        end = chart3[1]['data-tooltip'].find("points")

        WaterScarcity = chart3[1]['data-tooltip'][start:end]

        start = chart5[1]['data-tooltip'].find("MSI score: ") + len("MSI score: ")
        end = chart5[1]['data-tooltip'].find("points")

        AbioticResourceDepletion = chart5[1]['data-tooltip'][start:end]

        start = chart6[1]['data-tooltip'].find("MSI score: ") + len("MSI score: ")
        end = chart6[1]['data-tooltip'].find("points")

        Chemistry = chart6[1]['data-tooltip'][start:end]

    except:
        pass
    
    total_score = float(GlobalWarming) + float(Eutrophication) + float(WaterScarcity) + float(AbioticResourceDepletion) + float(Chemistry)
    
    df = pd.DataFrame([material_name, GlobalWarming, Eutrophication, WaterScarcity, 
                               AbioticResourceDepletion, Chemistry, total_score]).transpose()
    df.columns = ['Material','GlobalWarming','Eutrophication', 'WaterScarcity', 'AbioticResourceDepletion', 'Chemistry', 'Overall']
    driver.quit()
    
    return df


In [89]:
a = get_indexdata("https://msi.higg.org/sac-materials/detail/", keyword='199')


In [85]:
a

,Material,GlobalWarming,Eutrophication,WaterScarcity,AbioticResourceDepletion,Chemistry,Overall
0,Acrylic fabric,10.1,14.1,1.6,12.2,13.6,51.6


In [86]:
#aggregating all data together
df = pd.DataFrame()
for material in all_materials:
    for i in range(len(material)):
        table = get_indexdata(url="https://msi.higg.org/sac-materials/detail/", keyword=material[i])
        df = pd.concat([df,table],axis=0)


In [88]:
df.to_csv("msi-score.csv")

In [87]:
df

,Material,GlobalWarming,Eutrophication,WaterScarcity,AbioticResourceDepletion,Chemistry,Overall
0,"Acetate, Triacetate fabric",13.0,12.0,6.0,11.5,13.6,56.1
0,Acrylic fabric,10.1,14.1,1.6,12.2,13.6,51.6
0,Alpaca Fabric,71.1,186.8,3.0,8.3,13.6,282.8
0,Aramid fabric,10.2,33.3,2.4,11.3,13.6,70.8
0,Carbon fiber fabric,20.7,9.9,1.7,26.7,13.6,72.6
0,Cotton fabric,8.8,17.7,50.4,8.0,13.6,98.5
0,Elastane/Spandex fabric,10.2,6.4,2.9,11.2,13.6,44.3
0,Flax fiber fabric,18.1,111.2,3.3,15.6,13.6,161.8
0,Glass fiber fabric,5.9,6.0,0.6,5.2,13.6,31.3
0,Hemp fiber fabric,17.3,100.3,13.9,14.6,13.6,159.7
